In [400]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split,StratifiedKFold,GridSearchCV
from sklearn.metrics import roc_auc_score, auc, roc_curve,classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC,SVC
from sklearn import metrics, svm
from test_function import *
from sklearn.metrics import accuracy_score

In [401]:
column_list = ['chrom','pos','ID','ref','alt','qual','filter','info','pos-1']
clinvar = pd.read_csv('/storage/home/jkl5991/work/project/original_data/clinvar_20200113.vcf', sep='\t', header=None, comment='#', names = column_list)
clinvar.columns = column_list


In [333]:
clinvar

,chrom,pos,ID,ref,alt,qual,filter,info,pos-1,chr
0,1,865628,789256,G,A,.,.,"ALLELEID=707587;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.865628G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1",865627,chr1
1,1,874778,769497,GCCTCCCCAGCCACGGTGAGGACCCACCCTGGCATGATCCCCCTCATCA,G,.,.,"ALLELEID=777163;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.874817_874864del;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=Deletion;CLNVCSO=SO:0000159;GENEINFO=SAMD11:148398;MC=SO:0001575|splice_donor_variant;ORIGIN=1",874777,chr1
2,1,879481,780745,G,C,.,.,"ALLELEID=707599;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.879481G>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1",879480,chr1
3,1,880944,730521,G,A,.,.,"ALLELEID=732665;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.880944G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=NOC2L:26155;MC=SO:0001819|synonymous_variant;ORIGIN=1",880943,chr1
4,1,887409,735490,G,C,.,.,"ALLELEID=746686;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.887409G>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=NOC2L:26155;MC=SO:0001819|synonymous_variant;ORIGIN=1",887408,chr1
...,...,...,...,...,...,...,...,...,...,...
613698,MT,16011,690273,A,G,.,.,"ALLELEID=677665;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16011A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MT-TP:4571;ORIGIN=1",16010,chrMT
613699,MT,16013,690274,A,G,.,.,"ALLELEID=677666;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16013A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MT-TP:4571;ORIGIN=1",16012,chrMT
613700,MT,16015,690276,T,TATTCTCTGTTCTTTC,.,.,"ALLELEID=677667;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16018_16032dup;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=Duplication;CLNVCSO=SO:1000035;GENEINFO=MT-TP:4571;ORIGIN=1",16014,chrMT
613701,MT,16017,690275,T,C,.,.,"ALLELEID=677668;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16017T>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MT-TP:4571;ORIGIN=1",16016,chrMT


In [355]:
clinvar['chr'] = 'chr' + clinvar['chrom'].astype(str)
if len(clinvar['ref']) == len(clinvar['alt']):
    #clinvar['location'] = clinvar['chr'].str.cat(c[['pos','ref', 'alt']].astype(str), sep="-")
    

SyntaxError: unexpected EOF while parsing (<ipython-input-355-059e50565cac>, line 4)

In [351]:
clinvar[clinvar['ref'].str.len() == clinvar['alt'].str.len()]


,chrom,pos,ID,ref,alt,qual,filter,info,pos-1,chr
0,1,865628,789256,G,A,.,.,"ALLELEID=707587;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.865628G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1",865627,chr1
2,1,879481,780745,G,C,.,.,"ALLELEID=707599;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.879481G>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1",879480,chr1
3,1,880944,730521,G,A,.,.,"ALLELEID=732665;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.880944G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=NOC2L:26155;MC=SO:0001819|synonymous_variant;ORIGIN=1",880943,chr1
4,1,887409,735490,G,C,.,.,"ALLELEID=746686;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.887409G>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=NOC2L:26155;MC=SO:0001819|synonymous_variant;ORIGIN=1",887408,chr1
5,1,887989,730522,A,G,.,.,"ALLELEID=743794;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.887989A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=NOC2L:26155;MC=SO:0001627|intron_variant;ORIGIN=1",887988,chr1
...,...,...,...,...,...,...,...,...,...,...
613697,MT,16003,690272,T,C,.,.,"ALLELEID=677664;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16003T>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MT-TP:4571;ORIGIN=1",16002,chrMT
613698,MT,16011,690273,A,G,.,.,"ALLELEID=677665;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16011A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MT-TP:4571;ORIGIN=1",16010,chrMT
613699,MT,16013,690274,A,G,.,.,"ALLELEID=677666;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16013A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MT-TP:4571;ORIGIN=1",16012,chrMT
613701,MT,16017,690275,T,C,.,.,"ALLELEID=677668;CLNDISDB=MedGen:C0162671,OMIM:540000,Orphanet:ORPHA550,SNOMED_CT:39925003;CLNDN=Juvenile_myopathy,_encephalopathy,_lactic_acidosis_AND_stroke;CLNHGVS=NC_012920.1:m.16017T>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MT-TP:4571;ORIGIN=1",16016,chrMT


In [359]:
x_column = ['location','SIFT_pred','LRT_pred', 'MA_pred', 'PROVEN_pred', 'SLR_score', 'SIFT_score','LRT_omega', 
                'MA_score', 'PROVEN_score', 'Grantham', 'HMMEntropy','HMMRelEntropy', 'PredRSAB', 'PredRSAI', 
                'PredRSAE','PredBFactorF', 'PredBFactorM', 'PredBFactorS', 'PredStabilityH','PredStabilityM', 
                'PredStabilityL', 'PredSSE', 'PredSSH','PredSSC', 'dscore', 'phyloP_pri', 'phyloP_mam','phyloP_ver','RNA_seq','UNEECON']
y_column = 'clinvar_result'

def process(filename):
    path = '/storage/home/jkl5991/work/project/'
    source = path + filename
    file = pd.read_csv(source , sep = '\t')
    y = file.loc[:,[y_column]].values.flatten()
    X = file.loc[:,x_column]
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    loc = X_train['location'].str.split(pat = '-',expand = True).iloc[:,2]
    ref = file['location'].str.split(pat = '-',expand = True).iloc[:,3]
    alt = file['location'].str.split(pat = '-',expand = True).iloc[:,4]
    a = clinvar.loc[clinvar['pos'].isin(loc)]
    b = a.loc[a['ref'].isin(ref)]
    c = b.loc[b['alt'].isin(alt)]
    
    return(c, X_train, X_test, y_train, y_test)

In [363]:
domi_loc, X_train_domi, X_test_domi, y_train_domi, y_test_domi = process('dominant_std.tsv')
recess_loc, X_train_recess, X_test_recess, y_train_recess, y_test_recess = process('recessive_std.tsv')
unannotated_loc, X_train_unannotated, X_test_unannotated, y_train_unannotated, y_test_unannotated = process('unannotated_omit_std.tsv')

In [395]:
loc = domi['location'].str.split(pat = '-',expand = True).iloc[:,2]
ref = domi['location'].str.split(pat = '-',expand = True).iloc[:,3]
alt = domi['location'].str.split(pat = '-',expand = True).iloc[:,4]


a = clinvar.loc[clinvar['pos'].isin(loc)]
b = a.loc[a['ref'].isin(ref)]
c = b.loc[b['alt'].isin(alt)]
d = c.loc[c['info'].str.contains('dominant')]

print(a.shape)
print(b.shape)
print(c.shape)
print(d.shape)
print(domi.shape)
d

(2354, 9)
(2325, 9)
(2315, 9)
(1606, 9)
(2124, 33)


,chrom,pos,ID,ref,alt,qual,filter,info,pos-1
822,1,1273654,749017,C,G,.,.,"ALLELEID=745640;CLNDISDB=MedGen:C4225363,OMIM:616331|MedGen:CN517202;CLNDN=Robinow_syndrome,_autosomal_dominant_2|not_provided;CLNHGVS=NC_000001.10:g.1273654C>G;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=DVL1:1855;MC=SO:0001583|missense_variant;ORIGIN=1",NaN
989,1,1735987,224717,T,C,.,.,"ALLELEID=226496;CLNDISDB=.|.|Human_Phenotype_Ontology:HP:0001249,MedGen:C1843367|Human_Phenotype_Ontology:HP:0001250,MedGen:C0036572|Human_Phenotype_Ontology:HP:0001252,MedGen:C0026827|Human_Phenotype_Ontology:HP:0001263,MedGen:C0557874|Human_Phenotype_Ontology:HP:0002069,MedGen:C0494475|Human_Phenotype_Ontology:HP:0002376,MedGen:C1836830|Human_Phenotype_Ontology:HP:0002384,MedGen:C0270834|Human_Phenotype_Ontology:HP:0002474,MedGen:C0454641|Human_Phenotype_Ontology:HP:0010841,MedGen:C4021219|Human_Phenotype_Ontology:HP:0011198,MedGen:C4023476|MedGen:C4310774,OMIM:616973,Orphanet:ORPHA488613|MedGen:CN517202;CLNDN=Neurodevelopmental_Disability|hypotonia|Intellectual_disability|Seizures|Muscular_hypotonia|Global_developmental_delay|Generalized_tonic-clonic_seizures|Developmental_regression|Focal_seizures_with_impairment_of_consciousness_or_awareness|Expressive_language_delay|Multifocal_epileptiform_discharges|EEG_with_generalized_epileptiform_discharges|Mental_retardation,_autosomal_dominant_42|not_provided;CLNHGVS=NC_000001.10:g.1735987T>C;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Pathogenic/Likely_pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:139380.0005|UniProtKB_(protein):P62873",NaN
995,1,1737942,208722,A,G,.,.,"AF_EXAC=0.00001;ALLELEID=205216;CLNDISDB=.|.|Human_Phenotype_Ontology:HP:0000175,MedGen:C2981150,Orphanet:ORPHA2014|Human_Phenotype_Ontology:HP:0000486,MedGen:C0038379|Human_Phenotype_Ontology:HP:0000639,MedGen:C0028738|Human_Phenotype_Ontology:HP:0000821,MedGen:C0020676,Orphanet:ORPHA181396|Human_Phenotype_Ontology:HP:0001249,MedGen:C1843367|Human_Phenotype_Ontology:HP:0001250,MedGen:C0036572|Human_Phenotype_Ontology:HP:0001252,MedGen:C0026827|Human_Phenotype_Ontology:HP:0001263,MedGen:C0557874|Human_Phenotype_Ontology:HP:0001332,MedGen:C0013421|Human_Phenotype_Ontology:HP:0001508,MedGen:C2315100|Human_Phenotype_Ontology:HP:0001510,MedGen:C0456070|Human_Phenotype_Ontology:HP:0002376,MedGen:C1836830|Human_Phenotype_Ontology:HP:0002474,MedGen:C0454641|Human_Phenotype_Ontology:HP:0002509,MedGen:C1838391|Human_Phenotype_Ontology:HP:0002540,MedGen:C0560046|Human_Phenotype_Ontology:HP:0009062,MedGen:C3806604|Human_Phenotype_Ontology:HP:0010841,MedGen:C4021219|Human_Phenotype_Ontology:HP:0011198,MedGen:C4023476|Human_Phenotype_Ontology:HP:0200049,MedGen:C4021898|MeSH:D009190,MedGen:C3463824,OMIM:614286,Orphanet:ORPHA52688|MeSH:D030342,MedGen:C0950123|MedGen:C4310774,OMIM:616973,Orphanet:ORPHA488613|MedGen:CN517202;CLNDN=Neurodevelopmental_Disability|hypotonia|Cleft_palate|Strabismus|Nystagmus|Hypothyroidism|Intellectual_disability|Seizures|Muscular_hypotonia|Global_developmental_delay|Dystonia|Failure_to_thrive|Growth_delay|Developmental_regression|Expressive_language_delay|Limb_hypertonia|Inability_to_walk|Infantile_axial_hypotonia|Multifocal_epileptiform_discharges|EEG_with_generalized_epileptiform_discharges|Upper_limb_hypertonia|Myelodysplastic_syndrome|Inborn_genetic_diseases|Mental_retardation,_autosomal_dominant_42|not_provided;CLNHGVS=NC_000001.10:g.1737942A>G;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:139380.0002|UniProtKB_(protein):P62873",NaN
996,1,1737942,224715,A,T,.,.,"ALLELEID=226498;CLNDISDB=.|.|Human_Phenotype_Ontology:HP:0001250,MedGen:C0036572|Human_Phenotype_Ontology:HP:0001252,MedGen:C0026827|Human_Phenotype_Ontology:HP:0001263,MedGen:C0557874|Human_Phenotype_

In [311]:
clinvar.loc[clinvar['pos'] == 154490156]

,chrom,pos,ID,ref,alt,qual,filter,info
610861,X,154490156,218362,C,T,.,.,"ALLELEID=215090;CLNDISDB=MedGen:C0796195,OMIM:311510,Orphanet:ORPHA2379|MedGen:CN234611;CLNDN=Parkinsonism,_early_onset_with_mental_retardation|X-linked_dominant_Parkinson's_disease;CLNHGVS=NC_000023.10:g.154490156C>T;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:300774.0004;GENEINFO=RAB39B:116442;MC=SO:0001583|missense_variant;ORIGIN=1;RS=864309527"


In [114]:
pd.set_option('display.max_colwidth',10000)

In [250]:
import re
pattern = 'CLNSIG=Conflicting_interpretations_of_pathogenicity'
txt = clinvar['info']
count = 0 

if(re.search(pattern, txt)):
    count += 1
    print('yy')
else:
    print('nono')
    
print(count)

TypeError: expected string or bytes-like object

In [52]:
def check(listname):
    count = 0
    for i in (listname):
        print(i)
        if(clinvar['pos'] == i):
            info = clinvar[clinvar['pos'] == i]['info'].str.split(';')
            print(info)
            for item in info:
                print(item)
            break
#                 if(re.search(pattern, item)):
#                     count += 1
#                     print(item)
#                     print('')
#                 else:
#                     print(item)
#                     break

    #print(count)
            

In [392]:
def check(listname, check_length = 10000):
    count = 0
    for i in (listname):
        print(clinvar[clinvar['pos'] == i])
        print('yy')
        count += 1
        if(count == check_length):
            break
            

In [286]:
domi['location'].head(100)

0         chr1-1273653-1273654-C-G
1         chr1-1735986-1735987-T-C
2         chr1-1737941-1737942-A-G
3         chr1-1737941-1737942-A-T
4         chr1-1737947-1737948-T-C
                  ...             
95    chr1-244218457-244218458-A-G
96    chr1-244218465-244218466-C-T
97    chr1-244218466-244218467-G-A
98    chr1-244218519-244218520-C-T
99    chr1-244218540-244218541-G-T
Name: location, Length: 100, dtype: object

In [83]:
ref = domi['location'].str.split(pat = '-',expand = True).iloc[:,3]
alt = domi['location'].str.split(pat = '-',expand = True).iloc[:,4]

(1628, 8)
(2124, 33)


,chrom,pos,ID,ref,alt,qual,filter,info
822,1,1273654,749017,C,G,.,.,"ALLELEID=745640;CLNDISDB=MedGen:C4225363,OMIM:616331|MedGen:CN517202;CLNDN=Robinow_syndrome,_autosomal_dominant_2|not_provided;CLNHGVS=NC_000001.10:g.1273654C>G;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=DVL1:1855;MC=SO:0001583|missense_variant;ORIGIN=1"
920,1,1469423,721370,G,A,.,.,"ALLELEID=731742;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.1469423G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ATAD3A:55210;MC=SO:0001583|missense_variant;ORIGIN=1"
989,1,1735987,224717,T,C,.,.,"ALLELEID=226496;CLNDISDB=.|.|Human_Phenotype_Ontology:HP:0001249,MedGen:C1843367|Human_Phenotype_Ontology:HP:0001250,MedGen:C0036572|Human_Phenotype_Ontology:HP:0001252,MedGen:C0026827|Human_Phenotype_Ontology:HP:0001263,MedGen:C0557874|Human_Phenotype_Ontology:HP:0002069,MedGen:C0494475|Human_Phenotype_Ontology:HP:0002376,MedGen:C1836830|Human_Phenotype_Ontology:HP:0002384,MedGen:C0270834|Human_Phenotype_Ontology:HP:0002474,MedGen:C0454641|Human_Phenotype_Ontology:HP:0010841,MedGen:C4021219|Human_Phenotype_Ontology:HP:0011198,MedGen:C4023476|MedGen:C4310774,OMIM:616973,Orphanet:ORPHA488613|MedGen:CN517202;CLNDN=Neurodevelopmental_Disability|hypotonia|Intellectual_disability|Seizures|Muscular_hypotonia|Global_developmental_delay|Generalized_tonic-clonic_seizures|Developmental_regression|Focal_seizures_with_impairment_of_consciousness_or_awareness|Expressive_language_delay|Multifocal_epileptiform_discharges|EEG_with_generalized_epileptiform_discharges|Mental_retardation,_autosomal_dominant_42|not_provided;CLNHGVS=NC_000001.10:g.1735987T>C;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Pathogenic/Likely_pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:139380.0005|UniProtKB_(protein):P62873"
994,1,1737942,391609,A,C,.,.,"ALLELEID=364646;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.1737942A>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=GNB1:2782;MC=SO:0001583|missense_variant,SO:0001623|5_prime_UTR_variant;ORIGIN=1;RS=752746786"
995,1,1737942,208722,A,G,.,.,"AF_EXAC=0.00001;ALLELEID=205216;CLNDISDB=.|.|Human_Phenotype_Ontology:HP:0000175,MedGen:C2981150,Orphanet:ORPHA2014|Human_Phenotype_Ontology:HP:0000486,MedGen:C0038379|Human_Phenotype_Ontology:HP:0000639,MedGen:C0028738|Human_Phenotype_Ontology:HP:0000821,MedGen:C0020676,Orphanet:ORPHA181396|Human_Phenotype_Ontology:HP:0001249,MedGen:C1843367|Human_Phenotype_Ontology:HP:0001250,MedGen:C0036572|Human_Phenotype_Ontology:HP:0001252,MedGen:C0026827|Human_Phenotype_Ontology:HP:0001263,MedGen:C0557874|Human_Phenotype_Ontology:HP:0001332,MedGen:C0013421|Human_Phenotype_Ontology:HP:0001508,MedGen:C2315100|Human_Phenotype_Ontology:HP:0001510,MedGen:C0456070|Human_Phenotype_Ontology:HP:0002376,MedGen:C1836830|Human_Phenotype_Ontology:HP:0002474,MedGen:C0454641|Human_Phenotype_Ontology:HP:0002509,MedGen:C1838391|Human_Phenotype_Ontology:HP:0002540,MedGen:C0560046|Human_Phenotype_Ontology:HP:0009062,MedGen:C3806604|Human_Phenotype_Ontology:HP:0010841,MedGen:C4021219|Human_Phenotype_Ontology:HP:0011198,MedGen:C4023476|Human_Phenotype_Ontology:HP:0200049,MedGen:C4021898|MeSH:D009190,MedGen:C3463824,OMIM:614286,Orphanet:ORPHA52688|MeSH:D030342,MedGen:C0950123|MedGen:C4310774,OMIM:616973,Orphanet:ORPHA488613|MedGen:CN517202;CLNDN=Neurodevelopmental_Disability|hypotonia|Cleft_palate|Strabismus|Nystagmus|Hypothyroidism|Intellectual_disability|Seizures|Muscular_hypotonia|Global_developmental_delay|Dystonia|Failure_to_thrive|Growth_delay|Developmental_regression|Expressive_language_delay|Limb_hypertonia|Inability_to_walk|Infantile_axial_hypotonia|Multifocal_epileptiform_discharges|EEG_

In [242]:
c['chr'] = 'chr' + c['chrom'].astype(str)
c['pos-1'] = c['pos']-1

/storage/home/jkl5991/.conda/envs/sanEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/storage/home/jkl5991/.conda/envs/sanEnv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [243]:
c['location'] = c['chr'].str.cat(c[['pos-1', 'pos', 'ref', 'alt']].astype(str), sep="-")

/storage/home/jkl5991/.conda/envs/sanEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [245]:
x = domi.loc[~domi['location'].isin(c['location'])]
x

,location,accession_num,clinvar_result,SIFT_pred,LRT_pred,MA_pred,PROVEN_pred,SLR_score,SIFT_score,LRT_omega,...,PredStabilityL,PredSSE,PredSSH,PredSSC,dscore,phyloP_pri,phyloP_mam,phyloP_ver,RNA_seq,UNEECON
13,chr1-27102088-27102089-T-G,ENSG00000117713.13|ARID1A,0,1,1,0,1,1.157194,-0.630831,-0.029274,...,-1.461365,1.695227,-1.813271,-0.710107,0.849443,0.419929,0.666308,1.141196,-0.017472,1.797980
20,chr1-35250909-35250910-G-A,ENSG00000188910.7|GJB3,0,1,1,1,1,0.013358,-0.630831,-0.028450,...,-0.656494,0.671370,-0.580587,-0.668503,0.729638,0.459447,0.799135,1.683483,0.883651,0.087648
23,chr1-55505646-55505647-G-T,ENSG00000169174.9|PCSK9,0,0,0,0,0,1.824952,1.631479,-0.014845,...,-0.876003,1.019401,-1.004629,-0.668503,-1.790310,0.041939,-1.173234,-1.317590,0.826582,-0.796002
25,chr1-55509597-55509598-G-A,ENSG00000169174.9|PCSK9,0,0,1,0,0,0.968156,-0.177138,-0.028817,...,0.888210,-0.732891,0.928219,-0.086020,-0.391685,0.468036,-0.726520,-0.674182,0.695734,-0.767433
29,chr1-55509693-55509694-A-G,ENSG00000169174.9|PCSK9,1,1,0,1,1,-0.262290,-0.454281,-0.026773,...,1.770316,-0.947375,1.219134,-0.169230,0.345992,0.292787,0.240372,0.924953,0.671518,-0.449377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,chr3-172835372-172835373-T-C,ENSG00000144962.6|SPATA16,0,0,0,0,0,1.776267,3.204011,-0.023104,...,0.351628,-1.137577,1.155034,0.662886,-0.010890,-2.609144,-3.631647,-1.624027,-0.378574,-0.875694
2111,chr13-110833705-110833706-G-A,ENSG00000187498.10|COL4A1,0,0,0,0,0,0.013358,0.884220,-0.022622,...,-0.298773,-0.307970,0.282293,0.260696,-0.010890,0.468036,-0.998852,-1.132438,1.102618,-0.614051
2112,chr7-147675053-147675054-G-T,ENSG00000174469.13|CNTNAP2,0,0,1,0,0,0.013358,3.474996,-0.029274,...,-0.168693,0.440699,-0.398149,-0.391130,-1.749926,0.629541,1.000973,1.027191,-1.003780,-0.573829
2116,chr21-44479029-44479030-G-A,ENSG00000160200.13|CBS,0,0,0,0,0,-1.228278,-0.058156,-0.002761,...,-0.591453,0.319293,-0.200919,-0.529815,-0.010890,-2.420149,-2.158677,-3.704666,0.698928,-0.875963


In [249]:
clinvar.shape

(613703, 8)

In [247]:
domi.loc[domi['location'] == 'chr1-27102088-27102089-T-G']

,location,accession_num,clinvar_result,SIFT_pred,LRT_pred,MA_pred,PROVEN_pred,SLR_score,SIFT_score,LRT_omega,...,PredStabilityL,PredSSE,PredSSH,PredSSC,dscore,phyloP_pri,phyloP_mam,phyloP_ver,RNA_seq,UNEECON
13,chr1-27102088-27102089-T-G,ENSG00000117713.13|ARID1A,0,1,1,0,1,1.157194,-0.630831,-0.029274,...,-1.461365,1.695227,-1.813271,-0.710107,0.849443,0.419929,0.666308,1.141196,-0.017472,1.79798


### check(domi_loc2,50)

In [155]:
check(recess_loc2,50)

2800    AF_ESP=0.00069;AF_EXAC=0.00027;AF_TGP=0.00100;ALLELEID=172453;CLNDISDB=MedGen:C1837007,OMIM:609006|MedGen:CN169374|MedGen:CN517202;CLNDN=Deafness,_autosomal_recessive_36,_with_or_without_vestibular_involvement|not_specified|not_provided;CLNHGVS=NC_000001.10:g.6504598C>T;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ESPN:83715;MC=SO:0001583|missense_variant;ORIGIN=9;RS=115143295
Name: info, dtype: object

3019    AF_EXAC=0.00004;ALLELEID=75319;CLNDISDB=MedGen:C3809309,OMIM:615376;CLNDN=Charcot-Marie-Tooth_disease,_recessive_intermediate_c;CLNHGVS=NC_000001.10:g.6529456G>A;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:611101.0004;GENEINFO=PLEKHG5:57449;MC=SO:0001583|missense_variant;ORIGIN=1;RS=397515456
Name: info, dtype: object

3021    ALLELEID=16058;CLNDISDB=MedGen:C1970211,OMIM:611067|Or

In [164]:
check(unannotated_loc2,300)

0    ALLELEID=707587;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.865628G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1
Name: info, dtype: object

2    ALLELEID=707599;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.879481G>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1
Name: info, dtype: object

7    ALLELEID=746687;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.891393G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=NOC2L:26155;MC=SO:0001583|missense_variant;ORIGIN=1
Name: info, dtype: object

10    ALLELEID=707603;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g

540    ALLELEID=731528;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.986160C>T;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=AGRN:375790;MC=SO:0001819|synonymous_variant;ORIGIN=1
Name: info, dtype: object

541    AF_ESP=0.00324;AF_EXAC=0.00423;AF_TGP=0.00120;ALLELEID=249345;CLNDISDB=MedGen:CN169374|MedGen:CN517202;CLNDN=not_specified|not_provided;CLNHGVS=NC_000001.10:g.986165G>A;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Benign/Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=AGRN:375790;MC=SO:0001583|missense_variant;ORIGIN=1;RS=145444272
Name: info, dtype: object

544    AF_EXAC=0.00004;ALLELEID=447012;CLNDISDB=MedGen:C3808739,OMIM:615120;CLNDN=Myasthenic_syndrome,_congenital,_8;CLNHGVS=NC_000001.10:g.986190C>T;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GE

743    ALLELEID=731650;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.1168143G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=B3GALT6:126792;MC=SO:0001583|missense_variant;ORIGIN=1
Name: info, dtype: object

745    AF_ESP=0.00071;AF_EXAC=0.00121;AF_TGP=0.00020;ALLELEID=359211;CLNDISDB=MedGen:CN169374|MedGen:CN517202;CLNDN=not_specified|not_provided;CLNHGVS=NC_000001.10:g.1168173C>T;CLNREVSTAT=criteria_provided,_conflicting_interpretations;CLNSIG=Conflicting_interpretations_of_pathogenicity;CLNSIGCONF=Likely_benign(1)%3BUncertain_significance(1);CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=B3GALT6:126792;MC=SO:0001583|missense_variant;ORIGIN=1;RS=200646244
Name: info, dtype: object

746    AF_ESP=0.07707;AF_EXAC=0.10020;AF_TGP=0.10663;ALLELEID=265441;CLNDISDB=MedGen:CN169374;CLNDN=not_specified;CLNHGVS=NC_000001.10:g.1168180G>C;CLNREVSTAT=criteria_provided,_multiple_submitters,

958    ALLELEID=718341;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.1562548A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MIB2:142678;MC=SO:0001619|non-coding_transcript_variant,SO:0001819|synonymous_variant;ORIGIN=1
Name: info, dtype: object

959    ALLELEID=718342;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.1562768G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MIB2:142678;MC=SO:0001619|non-coding_transcript_variant,SO:0001819|synonymous_variant;ORIGIN=1
Name: info, dtype: object

961    ALLELEID=706825;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.1563198G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=MIB2:142678;MC=SO:0001583|missense_variant,SO:0001619|non-coding_transcript_

1570    AF_ESP=0.00008;AF_TGP=0.00399;ALLELEID=549520;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.2340056C>T;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=PEX10:5192;MC=SO:0001583|missense_variant;ORIGIN=1;RS=368273118
Name: info, dtype: object

1586    AF_ESP=0.00523;AF_EXAC=0.00159;AF_TGP=0.00719;ALLELEID=549522;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.2340159A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=PEX10:5192;MC=SO:0001583|missense_variant;ORIGIN=1;RS=35426403
Name: info, dtype: object

1593    AF_ESP=0.00008;AF_EXAC=0.00012;ALLELEID=281811;CLNDISDB=MedGen:C4721541,OMIM:214100,Orphanet:ORPHA912|MedGen:CN517202;CLNDN=Peroxisome_biogenesis_disorder_1A_(Zellweger)|not_provided;CLNHGVS=NC_000001.10:g.2340211C>T;CLNREVSTAT=criteria_provided,_conflicting_interpretations;CLNSIG=Conflic